# Grupo 03 - Desafío 01
## Estrategia de construcción
>**OBJETIVO:** Crear un df de post-producción que luego se transformará en producción para realizar análisis del dataset.
> * Se trabajará depurando cada columna del dataset con la finalidad de reconstruir datos perdidos y/o crear nuevas columnas que serán utilizadas a posteriori en la fase de análisis.

### Instalación de librerías

In [ ]:
#!pip install unidecode


### Importación de librerías

In [1]:
import sys
sys.path.append("../src")
import g3utils as g3

import pandas as pd
import numpy as np
import re as re
import unidecode

### Carga e inicialización de df

In [2]:
# Definición de df que permite registrar los valores recuperados de cada columna
df_recup = pd.DataFrame()

In [3]:
# Definición de filas a cargar en las pruebas
ca_filas = 40

In [4]:
# Carga del dataset en un df

df = pd.DataFrame(pd.read_csv("../data/properatti_minusculas.csv", encoding='UTF-8'))
#!head nombreArchivo.cvs --> comando de linux para ver las 5 primeras lineas. Se puede utilizar para ver como estan separados los datos

In [ ]:
# Para ver el contenido total de las columnas

pd.set_option('display.max_colwidth', -1)

In [5]:
# Reemplado de NaN's
df = g3.reemplaza_nan('sin datos',df)

In [ ]:
# Verificación del reemplazo
df.sample(1)

In [ ]:
df.columns.unique

## Armado inicial de df post-produccion
* Este bloque se reserva para realizar la precarga inicial dentro de un df de post-producción que finalmente se transformará en dataset de producción.

In [6]:
# Dadtaframe de posproducción. Tras depurar todas las regex, este DF será la base de análisis de negocio
# Se ejecuta una única vez y se le van incorporando las columnas depuradas.
# Tener en cuenta que el DF pospo definitivo tiene que tener la misma cantidad de filas que el DF base len(df)

pospo = g3.generar_df_posproduccion(ca_filas)
pospo.shape
#pospo

(40, 1)

## Procesamiento de limpieza y carga de columnas depuradas en df post-produccion 'pospo'
> *** Modo de uso:** 
>* En este markdown se vuelcan las funciones que generan y agregan columnas depuradas dentro de pospo
>* La codificación de desarrollo se realiza mas abajo dentro del título **Desarrollo** y una vez terminado se empaqueta dentro de una función consolidadora.



In [7]:
# Depuración de expresion 'ambientes' en 'description' y generacion de columna en pospro

def generar_ambientes(pospo):
    pattern = r'(\d*\w*)\s*amb'  # Patron para buscar metros ambientes
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas))
    dic = {'mono': 1, 'un':1, 'uno':1, 'dos':2, 'tres':3, 'cuatro':4, 'cinco':5 ,'seis':6 ,'siete':7}
    df_temp = g3.limpiar_columna_x_clave(dic, df1)
    pospo = g3.agregar_columna('ambiente', pospo, df_temp)
    return pospo


In [8]:
pospo = generar_ambientes(pospo)
pospo.shape

(40, 2)

In [9]:
# Depuración 'precio usd' en 'description' y agregado de columna en pospro

def generar_usd(pospo):
    pattern = r"(\d+\.?\,?\d+\.?\,?\d+)\s?u[$sd]\w?|u[$sd]\w?\s*(\d+\.?\,?\d+\.?\,?\d+)" # precio_usd
    df1 = g3.busca_claves(pattern,'description',df.head(ca_filas)) # Para trabajar con un DF acotado
    df2 = g3.obtener_df_indexado(df1, 'usd') # Genera DF acotado con los valores recuperados unicamente
    pospo = pd.merge(pospo, df2,on='indice', how='left')
    return pospo


In [10]:
pospo=generar_usd(pospo)
pospo.shape


(40, 3)

In [12]:
#pospo

# Desarrollo

In [ ]:
df_recup.shape

In [ ]:
df_recup.size


In [ ]:
df_recup.columns

In [ ]:
colum = 'amb'
valore = 7


In [ ]:
df_recup = registrar_recupero(valore, colum, df_recup)

In [ ]:
df_recup.shape

In [ ]:
df_recup

In [ ]:
#EN CONSTRUCCION


def registrar_recupero(valor, colu, df_recup):
    """
    Registra cantidad de valores recuperados de un feature en particular.
    Devuelve un data frame con los valores recuperados.
    Adicionalmente deja un archivo.csv a modo de syslog
    
    Parameters:
    -----------
    arg : valor, colu
    
    valor -- cant recuperada de datos del feature
    colu -- nombre de columna (feature)
    
    Returns:
    --------
    ret : data frame de una dimensión y archivo.csv
    
    """
    if df_recup.size == 0:     
        #recup = pd.DataFrame([valor], columns=[colu])
        df_recup = pd.DataFrame([valor], columns=[colu])
        print('primer dato en df')
    elif colu not in df_recup.columns:
        df_recup[colu] = [valor+1]
        print('columna creada')
    else:
        df_recup[colu] = [valor+2]
        print('columna existente')
    df_recup.to_csv(r'../data/valores_recuperados.csv', encoding='utf-8')
    return df_recup


In [ ]:
df_recup.T


In [ ]:
 df_recup.to_csv('../data/valores_recuperados.csv', encoding='utf-8')

### Patterns

In [ ]:
# Comentar lo que no se deba ejecutar
# Listado de todos los paterns configurados
# regex = re.compile(pattern, flags = re.IGNORECASE) # Se ejecuta dentro de funciones específicas

#pattern = r'(\d+)\s[m]2' # Patron para buscar metros cuadrados (m2)
#pattern = r'(\d*\w*)\s*amb'  # Patron para buscar metros ambientes
#pattern = r'(\d*)\s?\b[aA]mbientes\w*'
#pattern = r'(\d*\w*)\s*usd'
#pattern = r'(\d*\w*)\s*habitacion|(\d*\w*)\s?cuarto|(\d*\w*)\s?dorm|(\d*\w*)\s?pieza'
#pattern = r"(pileta)|(piscina)|(SUM)|(laundry)|(lavadero)|(terraza)|(solarium)|(baulera)|(sauna)|(gimnasio)|(salon de usos multiples)|(cochera)|(garage)|(gim)|(gym)" # amenities
#pattern = r"(casa)|(departamento)|(triplex)|(duplex)|(frente)|(contrafrente)|(PH)|(chalet)" # casa
#pattern = r"(\d*)\,?\d*\s?(?:m²|mts\s?2|metros\s?2|mts²|m2|metros\s?cuadrado|mts\s?cuadrado)" # superficie
pattern = r"(\d+\.?\,?\d+\.?\,?\d+)\s?u[$sd]\w?|u[$sd]\w?\s*(\d+\.?\,?\d+\.?\,?\d+)" # precio_usd
#pattern = r"(\d+\.?\,?\d+\.?\,?\d+)\s?\$|\$\s*(\d+\.?\,?\d+\.?\,?\d+)" # precio



### Regex

> Depuración sobre feature **'description'**

In [ ]:
# Ejecuta función de busqueda de claves en base a los Patterns previamente configurados

df1 = g3.busca_claves(pattern,'description',df.head(50000)) # Para trabajar con un DF acotado
#df1 = g3.busca_claves(pattern,'description',df) # Para trabajar con el DF completo
#df1.iloc[1][1]
#df1 = g3.reemplaza_nan('sin datos',df1)
#df1
#df1.iloc[2]
#len(df1.iloc[2])

In [ ]:
l1=g3.obtener_df_indexado(df1, 'usd')
#l1


In [ ]:
len(df)

### [FIN] Procesamiento de limpieza y carga de columna depurada en df post-produccion

### Procesamiento de armado intermedio de df de post-produccion
> * Modo de uso temporal hasta terminar de depurar todas las claves necesarias
> * Cada grupo de código se utiliza para realizar consolidado de filas dentro del DF que luego será el definitivo

### Armado de dataset definitivo

In [ ]:
# Agregado de columna limpia a DF definitivo
dfx = g3.agregar_columna('ambiente', dfx, df_temp)

# ------------------------------
# SIN USO
# ------------------------------

In [ ]:
#tmp_df = df["cases"].str.extract("([a-z])([0-9]+)([0-9]{2})",expand=True) 
tmp_df = df["description"].str.extract("((\d*)\s?\b[aA]mb\w*)",expand=True) 
tmp_df.dropna(axis=0)

In [ ]:
# TEST: Regex de prueba. Alimenta un DF con el resultado
# CONDICION: No lee NaN's
# NOTA: Me di cuenta que REGEX analiza todo el str y en muchos casos detecta la KEY en varias partes del 
#   str, ergo, al capturar el siguiente dato, lo incluye dentro de una lista y eso lo representa en otra columna
#   Entonces ponerle nombre a una única columna no tendría sentido. Si verificamos con .head(10) no vamos
#   a tener problemas porque justo las 10 primeras filas no tienen mas de una KEY en el str a analizar...

#m = pd.DataFrame([regex.findall(n) for n in df_a1['description'].head(10)], columns=["ambientes"] )
m = pd.DataFrame([regex.findall(n) for n in df_a1['description'].head(50)])
#m = pd.DataFrame([regex.findall(n) for n in df_a1['description']])
display(len(m))
#m
m[0]


In [ ]:
# Coloco nombre en la columna para luego hacer merge

df1.rename(columns={0:'ambientes'}, inplace=True)

### Reemplazo de NaN's
Esto se realiza porque la función de búsqueda explotaba 

In [ ]:
df_a1=df.replace(np.nan,'sin-dato')
#df_a1.head(2)

In [ ]:
def reemplaza_nan(df, clave):
    return df.replace(np.nan,clave)

### Funciones

In [ ]:
# Busca claves segun pattern en columna dentro de dataFrame 
# Entrada: pattern, columna, df
# Salida: DataFrame

def busca_claves(pattern, columna, df_aux):
    regex = re.compile(pattern, flags = re.IGNORECASE)
    m = pd.DataFrame([regex.findall(n) for n in df_aux[columna].head(10000)])
    return m[0]

In [ ]:
def existe_clave(key, columna, df_aux):    
    m = []
    for frase in df_aux[columna]:
        if key in frase:
            m.append(True)
        else:
            m.append(False)
    return sum(m)


In [ ]:
# No se está utilizando mas desde que el dataset se encuentra en minúsculas y sin acentos
def quitar_caracteres(column):
    for i in range(len(column)):
        if type(column[i]) == str:
            column[i] = str.lower(unidecode.unidecode(column[i]))

In [ ]:
# Cambia los literales numericos x floats. En caso de no detectar la clave en un diccionario, reemplaza por NaN.
# El reemplazo x NaN se hace para poder realizar operaciones.
# Mejoras: debiera recibir diccionario y lista_prop como parametros para dejar una función polimorfica. 
# Entrada: DataFrame
# Salida: DataFrame

def cambiar_x_nros(df):
    dic = {'mono': 1, 'un':1, 'uno':1, 'dos':2, 'tres':3, 'cuatro':4, 'cinco':5 ,'seis':6 ,'siete':7}
    lista_prop = ['mono','un','uno','dos','tres','cuatro','cinco','seis','siete']
    serie_1 = pd.Series([x if x not in lista_prop else dic.get(x) for x in df])
    serie_2 = pd.to_numeric(serie_1, errors='coerse', downcast='float') # coerse: pasa a NaN los no-numericos
    return pd.DataFrame(serie_2)

In [ ]:
quitar_caracteres(df["place_name"])
quitar_caracteres(df["place_with_parent_names"])
quitar_caracteres(df["country_name"])
quitar_caracteres(df["state_name"])
quitar_caracteres(df["description"])
quitar_caracteres(df["title"])


### Pruebas de campos

In [ ]:
# Para verificar el contenido de alguna fila al azar
df.sample(1)['description']

In [ ]:
# Para verificar algún problema en alguna fila en particular
df.iloc[293]['description']